In [1]:
import glob
import pickle
import os
import sys

if not os.path.join(os.path.abspath(''), '../') in sys.path:
    sys.path.append(os.path.join(os.path.abspath(''), '../'))
    
import numpy as np
import pandas as pd
from tqdm import notebook

from utils.feature_utils import filter_pose_noise

### specific to where I stored the csvs. you can change the data_root_dir

In [2]:
data_root_dir = '/Users/alexanderhsu/Documents/ACC_Study4_Fig4'
# sexes = ['Male', 'Female']
groups = ['Male', 'Female', 'Combined']

# groups = [f'Group{i}' for i in range(1, 8)]

conditions = ['Control', 'Experimental']

# conditions_extra = ['Condition3A', 'Condition3B']

### store all data in dictionary

In [3]:

filenames = {key: [] for key in groups}
data = {key: [] for key in groups}
for group in notebook.tqdm(groups):
    filenames[group] = {key: [] for key in conditions}
    data[group] = {key: [] for key in conditions}
    for condition in notebook.tqdm(conditions):
        
        filenames[group][condition] = glob.glob(str.join('/', 
                                                              (data_root_dir,
                                                               f'{group}', 
                                                               f'{condition}', 
                                                               '*.csv')))
        data[group][condition] = {key: [] for key in [f'file{i}' 
                                                           for i in range(len(filenames[group][condition]))]}
        for i, csv in enumerate(notebook.tqdm(filenames[group][condition])):
            temp_df = pd.read_csv(csv, header=[0, 1, 2, 3], sep=",", index_col=0)
            selected_pose_idx = np.arange(temp_df.shape[1])
            idx_llh = selected_pose_idx[2::3]
            # the loaded sleap file has them too, so exclude for both
            idx_selected = [i for i in selected_pose_idx if i not in idx_llh]
            currdf_filt, _ = filter_pose_noise(temp_df, idx_selected=idx_selected, idx_llh=idx_llh, 
                                               llh_value=0.1)
            data[group][condition][f'file{i}'] = currdf_filt

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [4]:
file_chosen = 0
filenames['Combined']['Control'][file_chosen]

'/Users/alexanderhsu/Documents/ACC_Study4_Fig4/Combined/Control/flfl-GFP_M3122_acA2040-120um__23670846__20221207_120605923DLC_resnet50_LUPE_MALEDec5shuffle1_350000.csv'

In [5]:
data['Male']['Experimental'][f'file{file_chosen}']

array([[641.48352158, 576.65801442, 642.56678367, ..., 406.21736968,
        599.1721015 , 368.149786  ],
       [648.4513616 , 570.46967983, 648.49591875, ..., 407.29971933,
        598.77419424, 366.08717871],
       [654.09782684, 560.24912113, 652.32407546, ..., 403.68559408,
        600.15012978, 361.54467559],
       ...,
       [649.81683826, 487.0004437 , 644.58509529, ..., 632.67028934,
        531.63663864, 650.3540678 ],
       [650.93240857, 483.62821817, 643.13291645, ..., 631.19939256,
        530.22387457, 648.32118094],
       [649.50445223, 487.503016  , 640.96755075, ..., 631.59388328,
        529.74590015, 652.39307439]])

In [6]:
with open('../processed_dataset/raw_data_study4.pkl', 'wb') as f:
    pickle.dump(data, f)